In [15]:
from datetime import datetime
import pandas as pd
import plotly.graph_objects as go
import yfinance as yf

In [17]:
# The trading algorithm will be tested and evaluated over three timelines:

# dcb = Dot Com Bubble
dcb_start = '1997-06-01'
dcb_end = '2002-12-01'

# crsh = 2008 Crash
crsh_start = '2007-06-01'
crsh_end = '2012-12-01'

# cvd = COVID-19
cvd_start = '2020-03-01'
cvd_end = '2022-06-01'


In [20]:
# Get price data from Yahoo! Finance for S&P 500, NASDAQ 100, and RUSSELL 2000
price_data = yf.download(
    '^GSPC ^NDX ^RUT', 
    start=dcb_start,
    end=cvd_end
)['Close']

# rename columns from tickers to descriptive names
price_data.rename(
    columns={
        '^GSPC': 'SP500', 
        '^NDX': 'NASDAQ100',
        '^RUT': 'RUSSELL2000'
    },
    inplace=True
)

price_data.dropna(inplace=True)

[*********************100%***********************]  3 of 3 completed


In [21]:
# Preview the data
display(price_data.head())
display(price_data.tail())

,SP500,NASDAQ100,RUSSELL2000
Date,,,
1997-06-02,846.359985,958.690002,383.519989
1997-06-03,845.479980,929.809998,383.279999
1997-06-04,840.109985,921.960022,382.670013
1997-06-05,843.429993,930.609985,384.989990
1997-06-06,858.010010,944.989990,387.140015


,SP500,NASDAQ100,RUSSELL2000
Date,,,
2022-05-24,3941.479980,11769.839844,1764.829956
2022-05-25,3978.729980,11943.929688,1799.160034
2022-05-26,4057.840088,12276.790039,1838.239990
2022-05-27,4158.240234,12681.419922,1887.900024
2022-05-31,4132.149902,12642.099609,1864.040039
